## Intoduction to the problem

<img src="attachment:f81c1a81-d971-4f8d-8d90-80d8b9685484.png" alt="satellite-symbol" width="120"/>

In the fight against climate change it is of utter importance to monitor carbon emissions and thus scientists are ablet to find main sources and potential patterns of carbon outputs. In contrast to Europe and North America, most of African countries do not have extensive systems to monitor such emissions from the ground.
In this competition open-source emission data of 2020 - 2021 from the Sentinel-5P satellite is analyzed and utilized to predict future carbon emissions in 2022.

The data consists of the following features resulting from the sentinel-5P measurements:

* Sulphur Dioxide - COPERNICUS/S5P/NRTI/L3_SO2
* Carbon Monoxide - COPERNICUS/S5P/NRTI/L3_CO
* Nitrogen Dioxide - COPERNICUS/S5P/NRTI/L3_NO2
* Formaldehyde - COPERNICUS/S5P/NRTI/L3_HCHO
* UV Aerosol Index - COPERNICUS/S5P/NRTI/L3_AER_AI
* Ozone - COPERNICUS/S5P/NRTI/L3_O3
* Cloud - COPERNICUS/S5P/OFFL/L3_CLOUD

The task is to build a model that is able to predict the emission target for each week at each location in 2022.


## Libraries 📖

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.ticker

## Load competition data 🗃️

In [ ]:
IMPORT_PATH = "./"
train_original = pd.read_csv(IMPORT_PATH + "train.csv", index_col='ID_LAT_LON_YEAR_WEEK')
test_original = pd.read_csv(IMPORT_PATH + "test.csv", index_col='ID_LAT_LON_YEAR_WEEK')

## Data exploration 🧭

First, let's have a look what data we are dealing with and what is the quality.

In [ ]:
train_original.info()

In [ ]:
train_original.head()

In [ ]:
# find week no of to be predicted data in 2022
train_original['week_no'].nunique()

In [ ]:
# find week no of to be predicted data in 2022
test_original['week_no'].nunique()

In [ ]:
missed_vals = train_original.isnull().sum()
missed_vals[missed_vals > 0].sort_values(axis=0, ascending=False).head(15)

**Conclusion**: UvAerosolLayerHeight_x has the most missing values, check how much percent are valid

In [ ]:
print((train_original['UvAerosolLayerHeight_aerosol_pressure'].notnull().sum() / 
       train_original['UvAerosolLayerHeight_aerosol_pressure'].size) * 100, "%")


⚠️ <span style="background-color:gold">This is valid for all UvAerosolLayerHeight_X and makes these features unusable thus it will be removed later on !</span> ⚠️

In [ ]:
# Features can be grouped (e.g. measured chemical compounds)
col_groups = []
for column in train_original.columns:
    if column.split('_')[0] not in col_groups:
        col_groups.append(column.split('_')[0])  

col_groups

In [ ]:
# Check invalid values per year
train_original.isnull().groupby(train_original['year']).sum().sum(axis=1)

In [ ]:
train_original['emission'].describe()

In [ ]:
train_grouped_by_location = train_original.groupby(['latitude','longitude'])
print("Unique locations: ", train_grouped_by_location.nunique().shape[0])
locations_mean_emmissions = train_grouped_by_location['emission'].mean().to_frame().reset_index(inplace=False)
locations_no_emissions = locations_mean_emmissions[locations_mean_emmissions['emission'] == 0]
print("Locations with no emissions ever: ", locations_no_emissions.shape[0])
locations_no_emissions

There are 497 unique locations in the dataframe of which 15 have no emissions at all.

In [ ]:
locations_mean_emmissions['emission'].sort_values(ascending=False).head(10)

 🔍 Also two locations have particular high mean emission values ! 🔍

In [ ]:
# First convert the date from the two features into pandas time stamp. Just take Monday as a random day in the specific week.
## before:
train_original[['year', 'week_no']].head(3)

In [ ]:
plot_data = train_original.copy(deep=True)
plot_data['date'] = pd.to_datetime(plot_data['year'].astype(str) + '.' + plot_data['week_no'].astype(str) +  '.Mon', format="%Y.%W.%a")

In [ ]:
# after:
plot_data[['year', 'week_no', 'date']].head(3)

Let's create diagrams of the emissions to see how they are distributed and what are some trends.

In [ ]:
plot_data['emission'].describe()

In [ ]:
# For the histogram plot very small values below 1 and really big values greater 1000 are cut off for visibility
X_MAX_PLOT = 1500
plt.figure(figsize=(8,4))
ax = sb.histplot(data=plot_data['emission'][(plot_data['emission'] >= 1) & (plot_data['emission'] <= X_MAX_PLOT)], 
                   bins=30, alpha=0.7, kde=True, stat='percent', line_kws={'color': "red", 'lw': 1.5, 'ls': '-'})
ax.set(xlabel='Emission value', ylabel='Emission measurements [%]')
ax.lines[0].set_color('coral')
ax.set_xticks(np.arange(0, X_MAX_PLOT, 150))

quant_50, quant_75, quant_95 = plot_data['emission'].quantile([0.50, 0.75, 0.95])
quantiles = [[quant_50, 0.8, " 50th", 31], [quant_75, 0.5, " 75th", 19], [quant_95, 0.3, " 95th", 11]]
for quantile in quantiles:
    plt.axvline(x=quantile[0], ymax=quantile[1] ,linestyle='--', color="slategrey", linewidth=1.0)
    plt.text(x=quantile[0], y=quantile[3], s=quantile[2])

plt.title("Distribution of emissions")
plt.show()

➡️ <b><span style="background-color:lightblue">Conclusion: Most of the emission values lie below 200</span><b>

In [ ]:
import matplotlib.patches as patches
import matplotlib.dates as mdates

# Not let's have a look at emissions per year
plt.figure(figsize=(10,4))
grouped_emissions = plot_data.groupby('date')['emission'].sum()
ax = sb.lineplot(data=grouped_emissions)

emission_mean = grouped_emissions.mean()
plt.axhline(y=emission_mean ,linestyle='-', color="coral", linewidth=1.0)
#draw rectangle to emphasize outlier
plt.axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-10-01'), color='salmon', alpha=0.1)
plt.axvline(pd.Timestamp('2020-03-01'), linestyle = "--", color='grey')
plt.axvline(pd.Timestamp('2020-10-01'), linestyle = "--", color='grey')

plt.title("Emissions over the year")
print("Mean value: ", emission_mean)

In [ ]:
# And create to heatmap of the averaged emissions per week for each year to find patterns
plt.figure(figsize=(14,3))
ax_hm = sb.heatmap(plot_data.pivot_table(index='year', columns='week_no', values='emission', aggfunc=np.mean), cmap="viridis")
ax_hm.hlines([1, 2], *ax_hm.get_xlim(), color="black", linewidth=1.0)
ax_hm.vlines(np.arange(start=0, stop=53, step=1), *ax_hm.get_ylim(),linestyle='--', color="black", linewidth=0.4)
plt.axvline(14, linestyle = "-", color='salmon')
plt.axvline(27, linestyle = "-", color='salmon')


In both the line plot (cumulated values over the year) and the heatmap (mean values per week) several outliers in the emissions, compared to 2019 and 2021, are clearly visible. Escpecially in the second quarter of 2020. This is likely linked to restrictive COVID-19 lockdown measures.

⚠️ <span style="background-color:gold">This specific time window needs to be handled in a proper way to prevent unwanted effects on our model later on.</span> ⚠️

Moreover, seasonal patterns can be identified. In 2019 and 2021 during 13-17th weeks and 39-44th weeks there are particular high emissions.

As further below already realized there are two locations with particular high mean measurements over the time.

In [ ]:
locations_mean_emmissions['emission'].sort_values(ascending=False)[0:2]

Let's find out where this two and the other emisison sources are

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(locations_mean_emmissions, 
                        lat="latitude", 
                        lon="longitude", 
                        color="emission",
                        color_continuous_scale=px.colors.sequential.Turbo,
                        size="emission",
                        zoom=7, 
                        height=500,
                        width=900)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

➡️ 
<b><span style="background-color:lightblue">The biggest source is near Nyungwe National Park. Potential reasons could be deforestation or wildfires. The second biggest source is near Karongi but it is hard to make assumptions regarding possible emitters in this area.</span><b>

### Summary of exploration 💡
* No features in the data are categorical but only numerical.
* The UvAerosolLayerHeight_X features miss nearly all values and makes these feature group unusable -> will be removed.
* Features belong to subgroups (e.g. measured chemical compounds).
* There are 497 unique locations in the dataframe of which 15 have no emissions at all.
* Most of the emission values lie below 200. The range is from 0 to 3167.
* The 2020 emissions deviate from 2019, 2021 and are generally recuded compared to these years. Especially in the second quarter of 2020 a great dip occurs. --> likely linked to restrictive COVID-19 lockdown measures.
* Excluding the effects of Covid seasonal patterns can be identified with peaks and valleys.
* Two locations have an extensively higher emission compared to all other unique locations. Near Nyungwe National Park and Karongi
* Train data consists of 53 weeks in all three years, test data of 49 weeks for 2022



## Pre-processing of data ⚙️

Add pandas timestamp like below already happened for the visualization data 

In [ ]:
train = train_original.copy(deep=True)
train['date'] = pd.to_datetime(train_original['year'].astype(str) + '.' + train_original['week_no'].astype(str) +  '.Mon', format="%Y.%W.%a")

As found out in the previous analysis the "**UvAerosolLayerHeight**" feature group has **less than 1 percent** of valid values. Due to this we drop these features all together.

In [ ]:
train.drop(columns=list(train.filter(regex='UvAerosolLayerHeight')), inplace=True)

The Covid-19 restrictions influence the emission data in 2020 so that it strongly deviates from a usual year. The many unique outliers do not help to train a generalised model and make predictions for "normal" years. An easy and straight forward way to compensate this is to scale the values of 2020 with the ratio between 2019/2021 and 2020.

In [ ]:
# calculate the week average of 2019, 2021
train_emission_no_covid = train[train['year'].isin((2019, 2021))]
train_emission_no_covid_avg = train_emission_no_covid.groupby('week_no')['emission'].mean()
# calculate the week average of 2020 aka covid restriction year
train_emission_covid = train[train['year'] == 2020]
train_emission_covid_avg = train_emission_covid.groupby('week_no')['emission'].mean()

In [ ]:
# calculate ratio between these means
ratios_each_week = train_emission_no_covid_avg / train_emission_covid_avg

In [ ]:
# And then scale the numbers of 2020 by replacing all week values by ratios and multiply it with 2020 emissions per week
rows_2020 = (train['year'] == 2020)
train.loc[rows_2020,'emission'] *= train['week_no'].map(ratios_each_week)

In [ ]:
# And create again a heatmap of the averaged emissions per week
fig, axs = plt.subplots(nrows=2)
fig.set_figwidth(12)
fig.set_figheight(4)

plots_hm = []
plots_hm.append(sb.heatmap(train_original.pivot_table(index='year', columns='week_no', values='emission', aggfunc=np.mean), cmap="viridis", ax=axs[0]))
plots_hm.append(sb.heatmap(train.pivot_table(index='year', columns='week_no', values='emission', aggfunc=np.mean), cmap="viridis", ax=axs[1]))

for hm in plots_hm:
    hm.hlines([1, 2], *ax_hm.get_xlim(), color="black", linewidth=1.0)
    hm.vlines(np.arange(start=0, stop=53, step=1), *ax_hm.get_ylim(),linestyle='--', color="black", linewidth=0.4)

➡️ 
<b><span style="background-color:lightblue">As one can see the pattern of 2020 now fits the other two years much better.</span><b>

### Adding features

#### Add holidays

In [ ]:
# to simplify only take the week_no of 2020 for holidays for all years,
# TODO  maybe do it right for all years
train['has_holidays'] = (train['week_no'].isin([0, 6-1, 15-1, 18-1, 27-1, 32-1, 33-1, 52-1]))

#### Add feature to take seasons into account
Add one hot encoding to data to differentiate betweens seasons

In [ ]:
def set_season(month):
    if 3 <= month <= 5:
        return 0 
    elif 6 <= month <= 8:
        return 1 
    elif 9 <= month <= 11:
        return 2
    else:
        return 3
        
train['season'] = train['date'].dt.month.apply(lambda month: set_season(month))